# Домашнее задание №5

Нужно реализовать собственный Transformer категориальных признаков. Код для проверки качества представлен в  скрипте transformer_checker.py, а шаблон, от которого стоит отталкиваться, в скрипте tranfomer_example.py. В скрипте обязательно должен быть класс CustomTransformer с реализацией вашей трансформации. Чекер вашего скрипта должен отработать за 20 секунд на машинке для проверки. Для сравнения на tranfomer_example.py чекер работает 2 секунды.

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy  as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,PolynomialFeatures,StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split

from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score

import dask_searchcv as dcv
from dask.diagnostics import ProgressBar

#### Чекер на example файле

In [20]:
%%time
!python transformer_checker.py

0.734744510209
CPU times: user 28.9 ms, sys: 36.3 ms, total: 65.2 ms
Wall time: 1.59 s


#### Чекер на моем файле

In [152]:
%%time
!python transformer_checker.py

0.905442730917
CPU times: user 28.8 ms, sys: 42.6 ms, total: 71.4 ms
Wall time: 1.49 s


#### Чекер на моем файле затюненый

In [36]:
%%time
!python transformer_checker.py

0.926611568966
CPU times: user 28.2 ms, sys: 41.8 ms, total: 70 ms
Wall time: 1.52 s


______

### Непосредственно решение

In [5]:
#Загрузка данных как в чекере
df=pd.read_csv("mushrooms.csv",header=None)
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
label_encoder = LabelEncoder()
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])
y = np.equal(y, 'p').astype(int)

In [6]:
lr=LogisticRegression(random_state=17)

In [7]:
#Обработка данных 1 (без обработки)

cross_val_score(lr,X,y,cv=3).mean()

0.81439504178786992

In [35]:
#Обработка данных 2 (OHE)

ohe=OneHotEncoder()
X_ohe=ohe.fit_transform(X)
cross_val_score(lr,X_ohe,y,cv=3).mean()

0.90544273091699889

### Тюнинг параметров логистической регресии

In [9]:
ohe=OneHotEncoder()
X_ohe=ohe.fit_transform(X)

lr=LogisticRegression(random_state=17)

In [ ]:
"solver":['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    "dual":[True,False],
        "multi_class":['ovr', 'multinomial'],

In [28]:
param_grid={"penalty":["l1","l2"],
            "tol":[0.000001,0.0000001],
            "C":np.linspace(6.3,6.6,20),
            "dual":[False],
            "solver":['liblinear'],
            "fit_intercept":[True,False],
            "warm_start":[True,False]}

In [29]:
grid=dcv.GridSearchCV(lr,param_grid,scoring='accuracy',cv=3,n_jobs=-1)
with ProgressBar():
    grid.fit(X_ohe,y)

[########################################] | 100% Completed | 31.3s


In [30]:
def best(grid):
    print(grid.best_params_)
    print(grid.best_score_)

In [31]:
best(grid)

{'C': 6.4578947368421051, 'dual': False, 'fit_intercept': False, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 1e-07, 'warm_start': False}
0.926637124569
